In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/undersampledrenn/Repeated_Edited_Nearest_Neighbors_Under-sampled_Dataset.csv


In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv('/kaggle/input/undersampledrenn/Repeated_Edited_Nearest_Neighbors_Under-sampled_Dataset.csv')
df = df[df['Label'] != 'BENIGN']

# Bi-GRU

Average Training Accuracy: 0.9224836826324463
Average Training Loss: 0.1946489095687866
Average Testing Accuracy: 0.9221948742866516
Average Testing Loss: 0.19666993319988252

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold

# Load the dataset
#df = pd.read_csv('data.csv')

# Split the data into features (X) and label (y)
X = df.drop('Label', axis=1).values
y = df['Label'].values

features = df.drop("Label", axis=1).values
labels = df["Label"].values

# One-hot encode the label
#y = tf.keras.utils.to_categorical(y)

encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = to_categorical(y)

# Define the number of folds for stratified K-Fold
n_splits = 5

# Initialize Stratified K-Fold object
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Define lists to store training and testing accuracies and losses for each fold
train_acc_list = []
train_loss_list = []
test_acc_list = []
test_loss_list = []

# Loop through each fold and train and evaluate the model
for i, (train_index, test_index) in enumerate(skf.split(X, y.argmax(1))):
    print("Running fold {}/{}...".format(i+1, n_splits))
    
    # Split the data into training and testing sets for the current fold
    train_X, test_X = X[train_index], X[test_index]
    train_y, test_y = y[train_index], y[test_index]
    
    # Define the model
    model = tf.keras.Sequential()

    # Add the first GRU layer
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=64, return_sequences=True), input_shape=(features.shape[1], 1)))

    # Add a dropout layer to prevent overfitting
    model.add(tf.keras.layers.Dropout(0.5))

    # Add the second GRU layer
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=32)))

    # Add a dropout layer to prevent overfitting
    model.add(tf.keras.layers.Dropout(0.5))

    # Add a dense layer for the output
    model.add(tf.keras.layers.Dense(units=train_y.shape[1], activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    # Train the model
    history = model.fit(train_X, train_y, batch_size=32, epochs=5, validation_split=0.2)

    # Evaluate the model on the training set
    train_loss, train_accuracy = model.evaluate(train_X, train_y)

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(test_X, test_y)

    # Append the training and testing accuracies and losses to their respective lists
    train_acc_list.append(train_accuracy)
    train_loss_list.append(train_loss)
    test_acc_list.append(test_accuracy)
    test_loss_list.append(test_loss)

# Print the average training and testing accuracies and losses across all folds
print("Average Training Accuracy: {}".format(np.mean(train_acc_list)))
print("Average Training Loss: {}".format(np.mean(train_loss_list)))
print("Average Testing Accuracy: {}".format(np.mean(test_acc_list)))
print("Average Testing Loss: {}".format(np.mean(test_loss_list)))


Running fold 1/5...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 42, 128)          25728     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 42, 128)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               31104     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 11)                715       
                                                                 
Total params: 57,547
Trainable param

# Bi-LSTM

In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, GRU

# Load the dataset
#df = pd.read_csv("data.csv")

# Split the dataset into features and labels
X = df.drop("Label", axis=1)
y = df["Label"]

features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)



model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(features.shape[1], 1)))
#model.add(Reshape((features.shape[1], 1)))  # add a reshape layer to make the input 3D
model.add(Bidirectional(LSTM(units=64, activation='tanh')))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=12, activation='relu'))
model.add(Dense(units=y.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model to the training data
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Epoch 1/5
4000/4000 [==============================] - 153s 37ms/step - loss: 0.5723 - accuracy: 0.7789 - val_loss: 0.4117 - val_accuracy: 0.8453
Epoch 2/5
4000/4000 [==============================] - 147s 37ms/step - loss: 0.3895 - accuracy: 0.8539 - val_loss: 0.3608 - val_accuracy: 0.8541
Epoch 3/5
4000/4000 [==============================] - 147s 37ms/step - loss: 0.2656 - accuracy: 0.9017 - val_loss: 0.2515 - val_accuracy: 0.9034
Epoch 4/5
4000/4000 [==============================] - 147s 37ms/step - loss: 0.2224 - accuracy: 0.9166 - val_loss: 0.2365 - val_accuracy: 0.9204
Epoch 5/5
1250/1250 [==============================] - 13s 10ms/step - loss: 0.2058 - accuracy: 0.9235
Test Loss: 0.20580387115478516
Test Accuracy: 0.9234904646873474


# Bi-GRU

In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam



# Extract the features and label
features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = to_categorical(labels)

# Define the number of folds
n_splits = 5

# Initialize lists to store results
train_acc_list = []
train_loss_list = []
test_acc_list = []
test_loss_list = []
history_Array=[]


model = tf.keras.Sequential()

# Add the first GRU layer
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=64, return_sequences=True), input_shape=(features.shape[1], 1)))

# Add a dropout layer to prevent overfitting
model.add(tf.keras.layers.Dropout(0.3))

# Add the second GRU layer
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=32)))

# Add a dropout layer to prevent overfitting
model.add(tf.keras.layers.Dropout(0.3))

# Add a dense layer for the output
model.add(tf.keras.layers.Dense(units=labels.shape[1], activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

# Split the data into training and testing sets
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
for train_index, test_index in skf.split(features,labels.argmax(1)):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Reshape the data for LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Train the model
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    history_Array.append(history)
    
    # Evaluate the model
    train_loss, train_accuracy = model.evaluate(X_train, y_train)

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    
    train_acc_list.append(train_accuracy)
    train_loss_list.append(train_loss)
    test_acc_list.append(test_accuracy)
    test_loss_list.append(test_loss)

print("Average Training Accuracy: {}".format(np.mean(train_acc_list)))
print("Average Training Loss: {}".format(np.mean(train_loss_list)))
print("Average Testing Accuracy: {}".format(np.mean(test_acc_list)))
print("Average Testing Loss: {}".format(np.mean(test_loss_list)))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_16 (Bidirecti  (None, 42, 128)          25728     
 onal)                                                           
                                                                 
 dropout_15 (Dropout)        (None, 42, 128)           0         
                                                                 
 bidirectional_17 (Bidirecti  (None, 64)               31104     
 onal)                                                           
                                                                 
 dropout_16 (Dropout)        (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 11)                715       
                                                                 
Total params: 57,547
Trainable params: 57,547
Non-trai

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout
from keras.utils import np_utils

# Extract the features and label
features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = np_utils.to_categorical(labels)

# Define the number of folds for stratified K-fold cross-validation
k_folds = 5

model = Sequential()
model.add(Bidirectional(GRU(units=64, return_sequences=True), input_shape=(features.shape[1], 1)))
model.add(Dropout(0.3))
model.add(Bidirectional(GRU(units=32)))
model.add(Dropout(0.3))
model.add(Dense(units=labels.shape[1], activation='softmax'))
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

train_acc_list = []
train_loss_list = []
test_acc_list = []
test_loss_list = []
# Perform stratified K-fold cross-validation
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
for i, (train_index, test_index) in enumerate(skf.split(features, labels.argmax(1))):
    print("Fold", i+1)

    # Split the data into training and testing sets
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Reshape the data for RNN
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Train the model
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

    # Evaluate the model
    train_loss, train_accuracy = model.evaluate(X_train, y_train)

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    
    train_acc_list.append(train_accuracy)
    train_loss_list.append(train_loss)
    test_acc_list.append(test_accuracy)
    test_loss_list.append(test_loss)

print("Average Training Accuracy: {}".format(np.mean(train_acc_list)))
print("Average Training Loss: {}".format(np.mean(train_loss_list)))
print("Average Testing Accuracy: {}".format(np.mean(test_acc_list)))
print("Average Testing Loss: {}".format(np.mean(test_loss_list)))

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_39 (Bidirecti  (None, 42, 128)          25728     
 onal)                                                           
                                                                 
 dropout_23 (Dropout)        (None, 42, 128)           0         
                                                                 
 bidirectional_40 (Bidirecti  (None, 64)               31104     
 onal)                                                           
                                                                 
 dropout_24 (Dropout)        (None, 64)                0         
                                                                 
 dense_65 (Dense)            (None, 11)                715       
                                                                 
Total params: 57,547
Trainable params: 57,547
Non-tra